In [ ]:
# Imports
import os, io, uuid, logging, re, threading, time, smtplib
from dotenv import load_dotenv
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import gradio as gr
from PIL import Image
import cv2
import numpy as np
from gtts import gTTS
import speech_recognition as sr
import google.generativeai as genai

In [ ]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("pawsense")

In [ ]:
# Load config
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise RuntimeError("Set GOOGLE_API_KEY in env")
genai.configure(api_key=API_KEY)
MODEL_NAME = "gemini-1.5-flash"
model = genai.GenerativeModel(MODEL_NAME)

In [ ]:
# Email Config
EMAIL_HOST = os.getenv("EMAIL_HOST")
EMAIL_PORT = int(os.getenv("EMAIL_PORT", 587))
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
EMAIL_SENDER = os.getenv("EMAIL_SENDER")

In [ ]:
# In-memory session store
SESSIONS = {}

DISCLAIMER = (
    "⚠ Disclaimer: I am an AI assistant for preliminary animal health guidance. "
    "This is NOT a substitute for a veterinarian. Always consult a vet for diagnosis or treatment."
)

In [ ]:
# Camera monitoring
camera_alert_flag = False
receiver_email_global = None

def monitor_camera(cam_index=0, check_interval=2, diff_threshold=0.6):
    global camera_alert_flag
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        logger.error("Camera not found!")
        camera_alert_flag = True
        send_email_alert("⚠ Camera not found!")
        return

    ret, prev_frame = cap.read()
    if not ret:
        logger.error("Unable to read from camera")
        camera_alert_flag = True
        send_email_alert("⚠ Unable to read from camera")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    while True:
        time.sleep(check_interval)
        ret, frame = cap.read()
        if not ret:
            logger.error("⚠ Camera disconnected!")
            camera_alert_flag = True
            send_email_alert("⚠ Camera disconnected!")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(prev_gray, gray)
        change_ratio = np.count_nonzero(diff) / diff.size

        if change_ratio > diff_threshold:
            logger.warning("⚠ Camera direction changed significantly!")
            camera_alert_flag = True
            send_email_alert("⚠ Camera direction changed significantly!")

        prev_gray = gray

In [ ]:
# Email Helper
def send_email_alert(message):
    global receiver_email_global
    if not receiver_email_global:
        logger.warning("⚠ No receiver email set, skipping email.")
        return
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = receiver_email_global
        msg['Subject'] = "🐾 PawSense Alert"
        msg.attach(MIMEText(message, "plain"))
        with smtplib.SMTP(EMAIL_HOST, EMAIL_PORT) as server:
            server.starttls()
            server.login(EMAIL_USER, EMAIL_PASS)
            server.sendmail(EMAIL_SENDER, receiver_email_global, msg.as_string())
        logger.info(f"✅ Email alert sent to {receiver_email_global}")
    except Exception as e:
        logger.error(f"❌ Failed to send email: {e}")

In [ ]:
def safe_text_from_response(response):
    if response is None:
        return ""
    if hasattr(response, "text"):
        return response.text or ""
    if getattr(response, "candidates", None):
        try:
            return response.candidates[0].content
        except Exception:
            return str(response)
    return str(response)

In [ ]:
def image_to_jpeg_bytes(pil_img: Image.Image, max_size=(1600,1600)):
    if pil_img.mode not in ("RGB", "RGBA"):
        pil_img = pil_img.convert("RGB")
    pil_img.thumbnail(max_size, Image.LANCZOS)
    buf = io.BytesIO()
    pil_img.save(buf, format="JPEG", quality=85)
    buf.seek(0)
    return buf.read()

URGENT_PATTERNS = [
    r"\bnot breathing\b", r"\bno breath\b", r"\bseizure\b", r"\bconvulsion\b",
    r"\bcollapse(ed|ing)?\b", r"\bbleeding (heavily|a lot|profusely)\b",
    r"\bcannot stand\b", r"\bwon't stand\b", r"\bsudden death\b",
]

In [ ]:
def is_urgent(text: str) -> bool:
    text = (text or "").lower()
    return any(re.search(p, text) for p in URGENT_PATTERNS)

In [ ]:
def call_gemini(conversation_messages, image_bytes=None):
    inputs = []
    for m in conversation_messages:
        role = m.get("role", "user")
        content = m.get("content", "")
        inputs.append(f"[{role.upper()}] {content}")
    if image_bytes:
        inputs.append({"mime_type": "image/jpeg", "data": image_bytes})
    try:
        response = model.generate_content(
            inputs,
            generation_config={"temperature": 0.2, "top_p": 0.95, "max_output_tokens": 512}
        )
        return safe_text_from_response(response)
    except Exception as e:
        logger.exception("Gemini call failed")
        return f"⚠ Error: failed to call model: {e}"

In [ ]:
# TTS
def text_to_audio(text, filename="reply.mp3"):
    tts = gTTS(text=text, lang="en")
    tts.save(filename)
    return filename

In [ ]:
# STT
def audio_to_text(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
    try:
        return r.recognize_google(audio)
    except Exception as e:
        logger.warning(f"Speech recognition failed: {e}")
        return ""


In [ ]:
# Video Analysis Helper
def analyze_video(video_path, species):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "⚠ Unable to process video file."
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(1, frame_count // 5)
    frames = []
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % sample_rate == 0:
            pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frames.append(pil_img)
        count += 1

    cap.release()

    insights = []
    for i, img in enumerate(frames):
        img_bytes = image_to_jpeg_bytes(img)
        prompt = f"Analyze visible signs of distress, injury, or abnormal behavior in this {species}. Be concise."
        response = model.generate_content(
            [{"role": "user", "content": prompt}, {"role": "system", "content": DISCLAIMER}],
            image=img_bytes,
            generation_config={"temperature": 0.2, "top_p": 0.9, "max_output_tokens": 300}
        )
        insights.append(safe_text_from_response(response))

    summary_prompt = (
        f"Summarize these frame-based findings into a concise veterinary observation for a {species}: "
        + " ".join(insights)
    )
    final_response = call_gemini([{"role": "user", "content": summary_prompt}])
    return final_response

In [ ]:
# Triage Logic
def triage_and_generate(session_id, user_text, pil_img, species):
    session = SESSIONS.setdefault(session_id, {"history": []})
    timestamp = datetime.utcnow().isoformat()
    session["history"].append({"role": "user", "content": f"Species: {species}. {user_text}", "time": timestamp})

    urgent_flag = is_urgent(user_text)

    system_prompt = (
        "You are an assistant that provides safe, evidence-based veterinary triage. "
        "Keep replies concise. If key facts are missing (species, age, vaccination, trauma), ask one follow-up. "
        "If urgent, respond with 'ESCALATE' and recommend immediate veterinary care."
    )

    conv = [{"role": "system", "content": system_prompt}, {"role": "system", "content": DISCLAIMER}]
    conv.extend(session["history"][-12:])

    img_bytes = None
    if pil_img:
        try:
            img_bytes = image_to_jpeg_bytes(pil_img)
            conv.append({"role": "user", "content": "Image attached: please describe visible signs and urgency."})
        except Exception as e:
            logger.exception("Image processing failed")
            session["history"].append({"role": "assistant", "content": f"⚠ Error processing image: {e}"})
            return format_chat(session["history"]), "", None

    reply_text = call_gemini(conv, image_bytes=img_bytes)
    if urgent_flag:
        reply_text = "⚠ ESCALATE: This appears urgent. " + reply_text
        send_email_alert(f"Urgent case detected!\n\n{user_text}")

    session["history"].append({"role": "assistant", "content": reply_text, "time": datetime.utcnow().isoformat()})
    audio_file = text_to_audio(reply_text)
    return format_chat(session["history"]), reply_text, audio_file

In [ ]:
# Format Chat
def format_chat(history):
    chat_display = []
    last_user = None
    for entry in history:
        role = entry.get("role", "user")
        content = entry.get("content", "")
        if role == "user":
            last_user = content
        elif role == "assistant":
            chat_display.append((last_user or "", content))
            last_user = None
        else:
            chat_display.append(("System", content))
    return chat_display

In [ ]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Row():
        gr.Markdown("""
        <div style="text-align: center; margin-bottom: 20px;">
            <h1 style="font-size: 2.2em; color: #4a5568;">🐾 PawSence</h1>
            <p style="font-size: 1.1em; color: #6b7280;">
                Your AI Vet Companion — Caring for Pets, Powered by AI
            </p>
        </div>
        """)

    with gr.Row(equal_height=True):
        with gr.Column(scale=2):
            chat = gr.Chatbot(label="💬 Conversation", height=460)
            with gr.Row():
                txt = gr.Textbox(placeholder="✍ Describe symptoms or ask a question...", lines=2, label="Type Symptoms")
                send = gr.Button("🚀 Send", variant="primary")

        with gr.Column(scale=1):
            gr.Markdown("### ⚙ Case Details")
            mode_toggle = gr.Radio(choices=["Text Mode", "Voice Mode"], value="Text Mode", label="Interaction Mode")
            species = gr.Dropdown(choices=["Dog","Cat","Horse","Cow","Goat","Sheep","Pig","Other"], label="Species", value="Select")
            email_input = gr.Textbox(label="📧 Receiver Email", placeholder="Enter your email to get alerts")
            img = gr.Image(type="pil", label="📷 Upload Image (optional)", height=180)
            video_input = gr.Video(label="🎞 Upload Video (optional)", height=180)
            video_btn = gr.Button("📊 Analyze Video", variant="secondary")

            with gr.Row():
                audio_in = gr.Audio(type="filepath", label="🎤 Speak Symptoms", sources=["microphone"], interactive=True, visible=False)
            audio_btn = gr.Button("🎙 Convert Audio → Text", variant="secondary", visible=False)

            clear_btn = gr.Button("🆕 Start New Session", variant="secondary")
            alert_box = gr.Textbox(label="📢 System Alerts", interactive=False)
            session_id_state = gr.State(str(uuid.uuid4()))

    # Events
    def user_submit(user_text, pil_img, sess_state, species_val, email_val):
        global receiver_email_global
        if email_val:
            receiver_email_global = email_val.strip()
        if not sess_state:
            sess_state = str(uuid.uuid4())
        chat_display, reply_text, audio_file = triage_and_generate(sess_state, user_text, pil_img, species_val)
        return chat_display, "", None, sess_state

    def reset_session():
        new_id = str(uuid.uuid4())
        SESSIONS[new_id] = {"history": []}
        return [], "", None, new_id

    def handle_audio(audio_path, sess_state, species_val):
        if not audio_path:
            return "", sess_state
        text = audio_to_text(audio_path)
        return text, sess_state

    def toggle_mode(mode):
        if mode == "Text Mode":
            return gr.update(visible=True), gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=True)

    def handle_video(video_path, species_val):
        if not video_path:
            return [("System", "⚠ No video uploaded.")]
        result = analyze_video(video_path, species_val)
        return [("🎞 Video Analysis Result", result)]

    def check_alerts():
        global camera_alert_flag
        if camera_alert_flag:
            return "⚠ ALERT: Camera disconnected or tampered!"
        return ""

    send.click(user_submit, inputs=[txt, img, session_id_state, species, email_input], outputs=[chat, txt, img, session_id_state])
    clear_btn.click(reset_session, outputs=[chat, txt, img, session_id_state])
    audio_btn.click(handle_audio, inputs=[audio_in, session_id_state, species], outputs=[txt, session_id_state])
    video_btn.click(handle_video, inputs=[video_input, species], outputs=[chat])
    mode_toggle.change(toggle_mode, inputs=[mode_toggle], outputs=[txt, send, audio_in, audio_btn])

    alert_timer = gr.Timer(value=3000, active=True)
    alert_timer.tick(check_alerts, outputs=[alert_box])

In [ ]:
# Start monitoring thread
monitor_thread = threading.Thread(target=monitor_camera, daemon=True)
monitor_thread.start()

In [ ]:
demo.launch(share=False, inline=True)